In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
#from . import fastai

In [2]:
#from fastai.imports import *
#from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [207]:
train_raw=pd.read_csv('train.csv',parse_dates=["DateTime"])
test_raw=pd.read_csv('test.csv',parse_dates=["DateTime"])

In [208]:
train_raw['Source']='train'
test_raw['Source']='test'
test_raw['is_click']='99'

In [209]:
data_raw=pd.concat([train_raw,test_raw])

In [210]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 128857
Data columns (total 16 columns):
DateTime                  592149 non-null datetime64[ns]
Source                    592149 non-null object
age_level                 568222 non-null float64
campaign_id               592149 non-null int64
city_development_index    432411 non-null float64
gender                    568222 non-null object
is_click                  592149 non-null object
product                   592149 non-null object
product_category_1        592149 non-null int64
product_category_2        150124 non-null float64
session_id                592149 non-null int64
user_depth                568222 non-null float64
user_group_id             568222 non-null float64
user_id                   592149 non-null int64
var_1                     592149 non-null int64
webpage_id                592149 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(6), object(4)
memory usage: 76.8+ MB


In [211]:
import re

def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)
        

In [212]:
#data_raw['Date']=pd.to_datetime(data_raw['DateTime'], infer_datetime_format=True)
add_datepart(data_raw, 'DateTime')
data_raw.head()

,Source,age_level,campaign_id,city_development_index,gender,is_click,product,product_category_1,product_category_2,session_id,...,DateTimeDay,DateTimeDayofweek,DateTimeDayofyear,DateTimeIs_month_end,DateTimeIs_month_start,DateTimeIs_quarter_end,DateTimeIs_quarter_start,DateTimeIs_year_end,DateTimeIs_year_start,DateTimeElapsed
0,train,4.0,359520,3.0,Female,0,C,4,NaN,140690,...,2,6,183,False,False,False,False,False,False,1498953600
1,train,2.0,105960,NaN,Female,0,C,5,NaN,333291,...,2,6,183,False,False,False,False,False,False,1498953600
2,train,2.0,359520,NaN,Female,0,C,4,NaN,129781,...,2,6,183,False,False,False,False,False,False,1498953600
3,train,3.0,359520,2.0,Male,0,I,3,NaN,464848,...,2,6,183,False,False,False,False,False,False,1498953600
4,train,2.0,405490,2.0,Male,0,C,3,NaN,90569,...,2,6,183,False,False,False,False,False,False,1498953660


In [213]:
data_raw.isnull().sum()/len(data_raw)*100

Source                       0.000000
age_level                    4.040706
campaign_id                  0.000000
city_development_index      26.975981
gender                       4.040706
is_click                     0.000000
product                      0.000000
product_category_1           0.000000
product_category_2          74.647597
session_id                   0.000000
user_depth                   4.040706
user_group_id                4.040706
user_id                      0.000000
var_1                        0.000000
webpage_id                   0.000000
DateTimeYear                 0.000000
DateTimeMonth                0.000000
DateTimeWeek                 0.000000
DateTimeDay                  0.000000
DateTimeDayofweek            0.000000
DateTimeDayofyear            0.000000
DateTimeIs_month_end         0.000000
DateTimeIs_month_start       0.000000
DateTimeIs_quarter_end       0.000000
DateTimeIs_quarter_start     0.000000
DateTimeIs_year_end          0.000000
DateTimeIs_y

In [214]:
NullCols = data_raw.isnull().sum()/len(data_raw)!=0

data_NullCols=data_raw.columns[NullCols]
print(data_NullCols)
#data_raw[data_NullCols].head()

Index(['age_level', 'city_development_index', 'gender', 'product_category_2',
       'user_depth', 'user_group_id'],
      dtype='object')


In [215]:
# TODO FOR LOOP
print("",data_NullCols[0],data_raw[data_NullCols[0]].isnull().sum()/len(data_raw)*100)

 age_level 4.04070597096


In [ ]:
#??data.drop
#data.drop('session_id',axis=1,inplace=True)

In [179]:
data_raw['gender'].dtypes

dtype('O')

In [216]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 128857
Data columns (total 28 columns):
Source                      592149 non-null object
age_level                   568222 non-null float64
campaign_id                 592149 non-null int64
city_development_index      432411 non-null float64
gender                      568222 non-null object
is_click                    592149 non-null object
product                     592149 non-null object
product_category_1          592149 non-null int64
product_category_2          150124 non-null float64
session_id                  592149 non-null int64
user_depth                  568222 non-null float64
user_group_id               568222 non-null float64
user_id                     592149 non-null int64
var_1                       592149 non-null int64
webpage_id                  592149 non-null int64
DateTimeYear                592149 non-null int64
DateTimeMonth               592149 non-null int64
DateTimeWeek             

In [217]:
data_na_filled = data_raw.fillna(method='ffill')

In [220]:
for a in data_na_filled.columns:
    var = data_na_filled[a].dtypes.name
    if (var == 'category'):
        print(data_na_filled[a].cat.categories)
        print (a ,"",data_na_filled[a].isnull().sum(),"",data_na_filled[a].dtypes)


In [221]:
X=data_na_filled

In [188]:
#y = X['is_click']

In [189]:
#X = X.drop('is_click',axis=1)

In [222]:
from sklearn import preprocessing
def convert(data):
    number = preprocessing.LabelEncoder()
    for azzx in data.columns:
        data[azzx] = number.fit_transform(data[azzx])
#    data['Source'] = number.fit_transform(data.Source)
#    data=data.fillna(-999)
    return data

train=convert(X)
#test=convert(y)

TypeError: '>' not supported between instances of 'str' and 'int'

In [223]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 128857
Data columns (total 28 columns):
Source                      592149 non-null int64
age_level                   592149 non-null int64
campaign_id                 592149 non-null int64
city_development_index      592149 non-null int64
gender                      592149 non-null int64
is_click                    592149 non-null object
product                     592149 non-null object
product_category_1          592149 non-null int64
product_category_2          592141 non-null float64
session_id                  592149 non-null int64
user_depth                  592149 non-null float64
user_group_id               592149 non-null float64
user_id                     592149 non-null int64
var_1                       592149 non-null int64
webpage_id                  592149 non-null int64
DateTimeYear                592149 non-null int64
DateTimeMonth               592149 non-null int64
DateTimeWeek                592

In [224]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                              random_state=0)


In [225]:
X_train = X[X['Source']==1]
X_test = X[X['Source']==0]


In [231]:
#X_train['is_click']

X_train['is_click']==99

Y_train = X_train[X_train['is_click']==1] 
#Y_test = X_test['is_click'==False] 


In [197]:
clf.fit(X_train,y)

ValueError: Unknown label type: 'unknown'

In [ ]:
clf.predict

In [ ]:
## List columns having null values
ZZ = data.isnull().sum().sort_index()/len(data)
ZZ[ZZ!=0]

In [ ]:
X.select_dtypes(exclude=['category']).columns

In [ ]:
X.select_dtypes(include=['category']).columns

In [ ]:
cat_Columns = X.select_dtypes(include=['category']).columns
print(cat_Columns.shape[0])

In [ ]:
sns.heatmap(X)

In [ ]:
plt.pcolor(X.isnull())

In [ ]:
import matplotlib.pyplot as plt
plt.spy(X.isnull())

##### List Non Null COlumns

In [ ]:
X.isnull().sum()

In [ ]:
X_Clean=X.dropna()

In [ ]:
NonNullCols = X.columns[X.isnull().any()==False]
NonNullCols = NonNullCols.drop(['DateTime', 'user_id'])


In [ ]:
print("Non Null COlumns ",NonNullCols)
print(NonNullCols.shape[0])

In [ ]:
X['product'].describe()

In [ ]:
charts = cat_Columns.shape[0]
for i in range(1,5):
    print(i,NonNullCols[i])
    plt.subplot(2, 3, i)
    plt.hist(x=NonNullCols[i],data=X)

In [ ]:
ZZ=X.columns

In [ ]:
sns.countplot(x="campaign_id", data=X, color="c");

In [ ]:
X.shape

In [ ]:
plt.hist(x=X.columns[3],data=X)

In [ ]:
X.columns[1]

In [ ]:
for i in range(2, 13):
    plt.subplot(4, 3, i-1)
    print(X.columns[i])
    plt.hist(x=X.columns[i],data=X)
#    plt.text(0.5, 0.5, str((2, 3, i)),
#             fontsize=18, ha='center')

In [ ]:
X.hist(column="campaign_id")

In [ ]:
fig = plt.figure(figsize = (15,12))
grid_size = (3,2)
hosts_to_fmt = []
for zz in X.columns:
    if(((zz!='DateTime') and (zz != 'user_id'))):
        print (zz)
        sns.countplot(y=zz, data=X, color="c");
        
#        print(X[zz].value_counts().T)
    


In [ ]:
X.corrplot()

In [ ]:
sns.pairplot(X)


In [ ]:
X.hist()

In [ ]:
X.dtypes(exclude=['bool']).T

In [ ]:
 X['user_type'].astype('category')

In [ ]:
X.columns

In [ ]:
X.hist()

In [ ]:
X_Clean.columns()

In [ ]:
X_Clean.hist()

In [ ]:
X_train['user_group_id'].hist()

In [ ]:
#plt.figure(figsize=(40,40)) 
# play with the figsize until the plot is big enough to plot all the columns
# of your dataset, or the way you desire it to look like otherwise

sns.heatmap(X.corr())

##### Count the Null Columns

In [ ]:
null_columns=X.columns[X.isnull().any()]
X[null_columns].isnull().sum()

In [ ]:
X.columns

In [ ]:
#plt.figure(figsize=(80,80)) 
hist = train.hist()

In [ ]:
train.columns

In [ ]:
pd.options.display.mpl_style = 'default'
train.boxplot()

### Perform the rain Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

### Train the Model

### Make the Prediction

### Compute tHe AUC_ROC curve